In [3]:
# import jepa etc

import torch
import torch.nn as nn
import torch.nn.functional as F
from models.jepa.jepa import JEPA
from models.time_series.patchTransformer import PatchTSTEncoder

In [4]:
DATASET_CONTEXT_LEN = 1048
DATASET_TARGET_LEN = 48

TRAIN_EPOCHS = 100
TRAIN_BATCH_SIZE = 64
EVAL_BATCH_SIZE = 256
LEARNING_RATE = 3e-4
PATCH_LEN = 8
PATCH_STRIDE = 8
LOG_EVERY = 500
CHECKPOINT_EVERY_EPOCHS = 5
AMP = True
GRAD_CLIP = 1.0

JEPA_D_MODEL = 192
JEPA_N_FEATURES = 9
JEPA_N_TIME_FEATURES = 2
JEPA_NHEAD = 3
JEPA_NUM_LAYERS = 3
JEPA_DIM_FF = 384
JEPA_DROPOUT = 0.1
JEPA_ADD_CLS = True
JEPA_POOLING = "mean"   # "cls" | "mean"
JEPA_PRED_LEN = 96

EMA_START = 0.99
EMA_END = 0.999
EMA_EPOCHS = 20

In [5]:
jepa_context_encoder = PatchTSTEncoder(
        patch_len=PATCH_LEN,
        d_model=JEPA_D_MODEL,  # transformer hidden dim
        n_features=JEPA_N_FEATURES,
        n_time_features=JEPA_N_TIME_FEATURES,
        nhead=JEPA_NHEAD,  # num of attention heads
        num_layers=JEPA_NUM_LAYERS,  # num of transformer blocks
        dim_ff=JEPA_DIM_FF,  # FFN hidden dim
        dropout=JEPA_DROPOUT,
        add_cls=JEPA_ADD_CLS,
        pooling=JEPA_POOLING,  # "cls" | "mean"
        pred_len=JEPA_PRED_LEN,
    )

jepa_target_encoder = PatchTSTEncoder(
        patch_len=PATCH_LEN,
        d_model=JEPA_D_MODEL,  # transformer hidden dim
        n_features=JEPA_N_FEATURES,
        n_time_features=JEPA_N_TIME_FEATURES,
        nhead=JEPA_NHEAD,  # num of attention heads
        num_layers=JEPA_NUM_LAYERS,  # num of transformer blocks
        dim_ff=JEPA_DIM_FF,  # FFN hidden dim
        dropout=JEPA_DROPOUT,
        add_cls=JEPA_ADD_CLS,
        pooling=JEPA_POOLING,  # "cls" | "mean"
        pred_len=JEPA_PRED_LEN,
    )

jepa_model = JEPA(
        jepa_context_encoder,
        jepa_target_encoder,
        d_model=JEPA_D_MODEL,
        ema_start=EMA_START,
        ema_end=EMA_END,
        n_epochs=EMA_EPOCHS,
    )

c:\Users\ollik\miniconda3\envs\.deeplearnenv\Lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [6]:
# print the model size
print(sum(p.numel() for p in jepa_model.context_enc.parameters() if p.requires_grad))

912001
